使用colab运行的配置代码

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110851 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive  -o nonempty

import os

os.chdir("drive/2048-api")


In [0]:
import keras
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import ExpectiMaxAgent
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,BatchNormalization
from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


定义Agent和model

In [0]:
from game2048.agents import Agent

class MyAgent(Agent):
    def __init__(self, game, display=None):
        self.game = game
        self.display = display
    def step(self):
       
        x_train=np.array(self.game.board)
        x=x_train
        x=np.log2(x+1)
        x=np.trunc(x)
        x = keras.utils.to_categorical(x, 12)
        x = x.reshape(1, 4, 4, 12)
        pred=model.predict(x,batch_size=128)
        r=pred[0]
        r1=r.tolist()
        direction2=r1.index(max(r1))
       
        
        return direction2            

In [0]:
model=Sequential()
model.add(Conv2D(filters= 128, kernel_size=(4,1),kernel_initializer='he_uniform', padding='Same', activation='relu',input_shape=input_shape))  # 第一个卷积层，32个卷积核，大小５x5，卷积模式SAME,激活函数relu,输入张量的大小 4*4*12
model.add(Conv2D(filters= 128, kernel_size=(1,4),kernel_initializer='he_uniform', padding='Same', activation='relu')) 

model.add(Conv2D(filters= 128, kernel_size=(1,1),kernel_initializer='he_uniform', padding='Same', activation='relu')) 
model.add(Conv2D(filters= 128, kernel_size=(2,2),kernel_initializer='he_uniform', padding='Same', activation='relu')) 
model.add(Conv2D(filters= 128, kernel_size=(3,3),kernel_initializer='he_uniform', padding='Same', activation='relu')) 
model.add(Conv2D(filters= 128, kernel_size=(4,4),kernel_initializer='he_uniform', padding='Same', activation='relu'))  

model.add(Flatten()) 
model.add(BatchNormalization())
model.add(Dense(256, kernel_initializer='he_uniform',activation='relu')) 
model.add(BatchNormalization())
model.add(Dense(128, kernel_initializer='he_uniform',activation='relu')) 
model.add(Dense(4, activation='softmax')) 

model.summary()
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics=['accuracy'])

In [0]:
model=load_model('model.h5')

主要训练部分

In [0]:
model=load_model('2048_new2_2048.h5')
#清空棋盘和方向的矩阵
count=0
while count<500:
    count=count+1
    print('第',count+1,'轮：')
    image=[]
    label=[]
    for _ in range(0,50):
        game = Game(4, score_to_win=1024, random=False)
        agent1 = ExpectiMaxAgent(game)

        while game.end==False:
            direction=agent1.step()
            
            x=np.array(game.board)
            x=np.log2(x+1)
            x=np.trunc(x)
            x = keras.utils.to_categorical(x, 12)
            x = x.reshape(1, 4, 4, 12)
            pred=model.predict(x,batch_size=128)
            r=pred[0]
            r1=r.tolist()
            mydirection=r1.index(max(r1))
            image.append(game.board)
            label.append(direction)
            game.move(mydirection)
            
        print(np.max(game.board))
        x=np.array(game.board)

    x_train=np.array(image)
    y_train=np.array(label)
    x_train=np.log2(x_train+1)
    x_train=np.trunc(x_train)
    x_train = keras.utils.to_categorical(x_train, 12)

    print(x_train.shape)
   
    y_train = keras.utils.to_categorical(y_train,4)
    
    model.train_on_batch(x_train, y_train)
    model.save('model.h5')


评测50次分数

In [0]:
i=[]

for _ in range(50):
    game = Game(4, random=False)

    agent1 = MyOwnAgent(game)
    agent1.play()
    i.append(np.max(game.board))
    
print(np.mean(i),np.max(i))

119.68 512.0


In [0]:
model.save('model666.h5')